In [5]:
import pandas as pd

In [25]:
#read from flat excel waste per category
df1_waste_cate = pd.read_excel('../source/waste_cate.xlsx', 'Sheet 1') 
df2_waste_cate = pd.read_excel('../source/waste_cate.xlsx', 'Sheet 2')
#read from flat excel food waste 
df1_food_waste = pd.read_excel('../source/food_waste.xlsx','Sheet 1')
df2_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 2')
df3_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 3')
df4_food_waste  = pd.read_excel('../source/food_waste.xlsx','Sheet 4')


#read from bigquery csv
df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')

#read from API csv
df_api_12 = pd.read_csv('../source/api_data12.csv')
df_api_12_3 = pd.read_csv('../source/api_data12_3.csv')



c:\Users\krant\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\krant\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\krant\AppData\Local\Temp\ipykernel_26224\1639646460.py:11: DtypeWarning: Columns (9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bq_keywords = pd.read_csv('../source/bq_keywords_food_waste.csv')


In [88]:
#read from flat excel waste per category
 

# Slice rows from index 8 to 48 of df1_waste_cate and reset the index, dropping the old index
df_waste = df1_waste_cate[8:49].reset_index(drop=True)

# Rename the columns of df_waste using the values from the 9th row of df1_waste_cate
df_waste = df_waste.rename(columns=df1_waste_cate.iloc[7])

# Add a new column named 'year' to df_waste and set all values in that column to 2020
df_waste['year'] = 2020

# Slice rows from index 8 to 48 of df2_waste_cate and reset the index, dropping the old index
df_waste2 = df2_waste_cate[8:49].reset_index(drop=True)

# Rename the columns of df_waste2 using the values from the 9th row of df2_waste_cate
df_waste2 = df_waste2.rename(columns=df2_waste_cate.iloc[7])

# Add a new column named 'year' to df_waste2 and set all values in that column to 2021
df_waste2['year'] = 2021

# Concatenate df_waste and df_waste2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_waste = pd.concat([df_waste, df_waste2]).reset_index(drop=True)


df_waste

,COICOP (Labels),Total,NaN,Food and non-alcoholic beverages,NaN,Food,NaN,Non-alcoholic beverages,NaN,"Alcoholic beverages, tobacco and narcotics",...,NaN,Social protection,NaN,Insurance,NaN,Financial services n.e.c.,NaN,Other services n.e.c.,NaN,year
0,GEO (Labels),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
1,European Union - 27 countries (from 2020),100,NaN,14.8,NaN,13.4,NaN,1.4,NaN,4.5,...,NaN,86944.5,NaN,184448.1,NaN,168105.4,NaN,:,NaN,2020
2,Euro area - 19 countries (2015-2022),100,NaN,14.3,NaN,13,NaN,1.3,NaN,4.3,...,NaN,77863,NaN,170124.5,NaN,139379.6,NaN,:,NaN,2020
3,Belgium,100,NaN,14.2,NaN,13.1,NaN,1.1,NaN,4.4,...,NaN,2510.9,NaN,6691.3,NaN,7746.5,NaN,4539.9,NaN,2020
4,Bulgaria,100,NaN,20.7,NaN,19,NaN,1.7,NaN,5.7,...,NaN,4.6,NaN,255.7,NaN,236.1,NaN,392.6,NaN,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,North Macedonia,100,NaN,30.5,NaN,27.2,NaN,3.4,NaN,5.2,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,2021
78,Albania,:,NaN,:,NaN,:,NaN,:,NaN,:,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,2021
79,Serbia,100,NaN,23.2,NaN,20.6,NaN,2.6,NaN,8.5,...,NaN,77.9,NaN,255.7,NaN,893.2,NaN,239.3,NaN,2021
80,Türkiye,100,NaN,23.3,NaN,:,c,:,c,3,...,NaN,:,NaN,:,NaN,:,NaN,:,NaN,2021


In [83]:
# Slice rows from index 10 to 38 of df1_food_waste and reset the index, dropping the old index
df_kilo_capita = df1_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita using the values from the 9th row of df1_food_waste
df_kilo_capita = df_kilo_capita.rename(columns=df1_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita and set all values in that column to 2020
df_kilo_capita['year'] = 2020

# Slice rows from index 10 to 38 of df2_food_waste and reset the index, dropping the old index
df_kilo_capita2 = df2_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_kilo_capita2 using the values from the 9th row of df2_food_waste
df_kilo_capita2 = df_kilo_capita2.rename(columns=df2_food_waste.iloc[9])

# Add a new column named 'year' to df_kilo_capita2 and set all values in that column to 2021
df_kilo_capita2['year'] = 2021

# Concatenate df_kilo_capita and df_kilo_capita2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_kilo_capita = pd.concat([df_kilo_capita, df_kilo_capita2]).reset_index(drop=True)


In [84]:
# Slice rows from index 10 to 38 of df3_food_waste and reset the index, dropping the old index
df_tonne = df3_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_tonne using the values from the 9th row of df3_food_waste
df_tonne = df_tonne.rename(columns=df3_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne and set all values in that column to 2020
df_tonne['year'] = 2020

# Slice rows from index 10 to 38 of df4_food_waste and reset the index, dropping the old index
df_tonne2 = df4_food_waste[10:39].reset_index(drop=True)

# Rename the columns of df_tonne2 using the values from the 9th row of df4_food_waste
df_tonne2 = df_tonne2.rename(columns=df4_food_waste.iloc[9])

# Add a new column named 'year' to df_tonne2 and set all values in that column to 2021
df_tonne2['year'] = 2021

# Concatenate df_tonne and df_tonne2 vertically, stack them on top of each other, and reset the index, dropping the old index
df_tonne = pd.concat([df_tonne, df_tonne2]).reset_index(drop=True)

